# Conexión a Azure Quantum y envío del trabajo

Suponemos que ya hemos realizado todos los pasos para usar AzureQPU:

    - Crear una cuenta en Azure
    - Suscribir un programa gratuito o de pago
    - Crear un servicio de Azure Quantum creando previamente un espacio de trabajo. 

Hecho esto ya solo tendría que establecer la conexión mediante mi identificador de recurso y localización de ejecución. 

    - MyResourceID
    - MyLocation


**Nota:** En el documento CREACIÓN DE CUENTA AZURE QUANTUM se explica cómo crear un área de trabajo:

https://mega.nz/file/MiMlCKJA#l1VMMoEwtq3UHLlgiCJiu4nSMEgS31V_cw0zY2tJobQ

## Instalación del Azure Quantum Developer Kit 


Podemos instalar el QDK de Azure Quantum en Python para Q#, Qiskit, Cirq:

https://learn.microsoft.com/es-es/azure/quantum/install-overview-qdk

Para Cirq:

%pip install qsharp azure-quantum[cirq]    

## Envío de un programa Cirq a Azure Quantum

https://learn.microsoft.com/es-es/azure/quantum/quickstart-microsoft-cirq?tabs=tabid-ionq&pivots=platform-local

In [ ]:
%pip install qsharp azure-quantum[cirq]

In [2]:
import azure.quantum
from azure.quantum.cirq import AzureQuantumService

In [6]:
workspace = azure.quantum.Workspace(
    resource_id = "/subscriptions/0f4027d3-276d-4941-ad53-a9244c367728/resourceGroups/imardiegoAzureQuantumResourcesGroup/providers/Microsoft.Quantum/Workspaces/imardiegoAzureQuantumWorkArea",
    location = "westeurope"
)

In [10]:
targets = workspace.get_targets()
print("This workspace's targets:")
targets

This workspace's targets:


[<Target name="quantinuum.sim.h1-1sc", avg. queue time=8 s, TargetAvailability.AVAILABLE>,
 <Target name="quantinuum.sim.h1-1e", avg. queue time=18 s, TargetAvailability.AVAILABLE>,
 <Target name="rigetti.sim.qvm", avg. queue time=5 s, TargetAvailability.AVAILABLE>]

In [11]:
import cirq

q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5,             # Square root of X
    cirq.CX(q0, q1),              # CNOT
    cirq.measure(q0, q1, key='b') # Measure both qubits
)
print(circuit)

0: ───X^0.5───@───M('b')───
              │   │
1: ───────────X───M────────


In [ ]:
result = service.run(program=circuit, repetitions=100, target=targets[2].name)

NameError: name 'service' is not defined

In [ ]:
result = service.run(
    program=circuit,
    repetitions=100,
    target="ionq.qpu.aria-1",
    timeout_seconds=500 # Set timeout to accommodate queue time on QPU
)

In [ ]:
print(result)

Otra versión

In [12]:
import cirq
from qiskit import QuantumCircuit, transpile
from azure.quantum.qiskit import AzureQuantumProvider

# --- 1. Circuito en Cirq ---
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5,             # √X
    cirq.CX(q0, q1),             # CNOT
    cirq.measure(q0, q1, key='b')
)

# --- 2. Exportar a QASM ---
qasm_str = cirq.qasm(circuit)

# --- 3. Importar en Qiskit ---
qc = QuantumCircuit.from_qasm_str(qasm_str)

print("Circuito en Qiskit:")
print(qc)

# --- 4. Conectarse a Azure Quantum ---
# Sustituye estos valores por los de tu workspace
resource_id = "<TU_RESOURCE_ID>"
location = "<TU_LOCATION>"

provider = AzureQuantumProvider(resource_id=resource_id, location=location)

# Ver lista de backends disponibles
for backend in provider.backends():
    print(backend.name())

# --- 5. Elegir backend (QPU) ---
backend = provider.get_backend("rigetti.qpu.aspen-m-3")  # Ejemplo

# --- 6. Transpilar y enviar trabajo ---
compiled = transpile(qc, backend)
job = backend.run(compiled, shots=100)  # shots = número de ejecuciones

# --- 7. Obtener resultados ---
result = job.result()
print("Resultados:")
print(result.get_counts())


ModuleNotFoundError: No module named 'qiskit'

In [ ]:
# Primero instala los paquetes necesarios si no los tienes:
# pip install azure-quantum[cirq]

import cirq
from azure.quantum.cirq import AzureQuantumService

# --- 1. Define tus qubits y circuito en Cirq ---
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5,             # √X gate
    cirq.CX(q0, q1),             # CNOT
    cirq.measure(q0, q1, key='b') # Medición de ambos qubits
)

print("Circuito local:")
print(circuit)

# --- 2. Configura el servicio de Azure Quantum ---
resource_id = "<TU_RESOURCE_ID>"  # p.ej "/subscriptions/.../resourceGroups/.../providers/Microsoft.Quantum/Workspaces/..."
location = "<TU_LOCATION>"         # p.ej "eastus", "westus", "westeurope", etc.

service = AzureQuantumService(resource_id=resource_id, location=location, default_target="<NOMBRE_DEL_TARGET>")

# Puedes ver los targets disponibles con:
print("Targets disponibles:")
targets = service.targets()
for t in targets:
    print(f"  {t.name}, estado: {t.availability}")

# --- 3. Enviar el circuito para ejecutarlo ---
# Repeticiones (shots) o repeticiones según el target
reps = 100  

# target real; debe ser uno de los que el workspace provee, p.ej "ionq.qpu", "honeywell.hqs-lt-s1-apival", etc.

job = service.run(program=circuit, repetitions=reps, target="<NOMBRE_DEL_TARGET>")

# --- 4. Esperar al completado y obtener resultados ---
# Nota: puede tomar un tiempo dependiendo del QPU, cola, etc.
result = job.results()

print("Resultado del job:")
print(result)  # verás algo como resultado de las mediciones
print("Cadena de bits medidos 'b':")
print(result.measurements['b'])


### Envío de programa Cirq a través del simulador

https://learn.microsoft.com/es-es/azure/quantum/quickstart-microsoft-cirq?tabs=tabid-ionq&pivots=platform-simulator

**Hacerlo!!!!**